## Reading packages and data

In [2]:
# Load packages
import numpy as np
import pandas as pd
import json
import nltk
import io 
import os
import gzip
from tqdm import tqdm
from functools import partial 
import copy
import sys

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pickle

# spacy for lemmatization
import spacy

import itertools
import collections

import itertools as it
import re
import time
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/virenbajaj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/virenbajaj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/virenbajaj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# def read_pdf(file_name, start_line, end_line, ids):
#     papers_ids_text = []; abstract = []; body_text = []; whole_text = []

#     with open(file_name) as f:
#         for _ in range(start_line):
#             next(f)
#         index = 0
#         for line in f:
#             paper = json.loads(line)
# #             if index > end_line - start_line:
# #                 break
# #             index += 1
#             if paper['paper_id'] in ids:
#                 print('in')
#                 papers_ids_text.append(paper['paper_id'])
#                 if paper['abstract']:
#                     print(paper['abstract'])
#                     abstract.append(paper['abstract'][0]['text'])
#                 else: 
#                     abstract.append('')
#                 text = []
#                 full_text = ''
#                 if paper['body_text']:
#                     for entry in paper['body_text']:
#                         if entry['section'] and entry['text']:
#                             section = {key: entry[key] for key in ['section', 'text']}
#                             text.append(section)
#                             if full_text:
#                                 full_text = full_text + '\n' + entry['text']
#                             else:
#                                 full_text = entry['text']
#                     body_text.append(text)
#                     whole_text.append(full_text)
#                 else:
#                     body_text.append([])
#                     whole_text.append('')
                
#         textdata = pd.DataFrame({'paper_id': papers_ids_text, 'abstract': abstract, 'body_text': body_text, 'whole_text': whole_text})

#         return textdata

## Preprocessing

Uncompressing papers from fields we want, saving the metadata and pdf_parses as pickled dictionaries

In [42]:
def process_metadata(metadata_file,fields=None, ids=None,output_dir = './processed/'):
    # Go through metadata files to get relevant paper ids and titles
    ids = []; title = []; field = [];
    # if file is compressed
    if metadata_file[-3:] == '.gz':
        output_file = metadata_file[:-3]
        gz = gzip.open(metadata_file, 'rb')
        f = io.BufferedReader(gz)
        f_out = open(output_file,'wb')
    else:
        f = open(metadata_file)
        f_out = None

    for line in tqdm(f.readlines()):
        paper = json.loads(line)
        if not fields:
            if not ids:
                ids.append(paper['paper_id'])
                title.append(paper['title'])
                field.append(paper['mag_field_of_study'])
                if f_out:
                    f_out.write(line)
            elif paper['paper_id'] in ids:
                print('id found: ', paper['paper_id'])
                
                ids.append(paper['paper_id'])
                title.append(paper['title'])
                field.append(paper['mag_field_of_study'])
                if f_out:
                    f_out.write(line)
        elif paper['mag_field_of_study']:
            field_in = any([x in fields for x in paper['mag_field_of_study']])
            if field_in:
                ids.append(paper['paper_id'])
                title.append(paper['title'])
                field.append(paper['mag_field_of_study'])
                if f_out:
                    f_out.write(line)
    f.close()
    if f_out:
        f_out.close()
    # create and save dataframe in output_dir/meta_df
    meta_df = pd.DataFrame({'ids':ids, 'titles':title, 'field': field})
    meta_df_dir = output_dir + 'meta_df/'
    os.makedirs(meta_df_dir, exist_ok=True)
    file_name_without_path_or_ext = metadata_file.split('/')[-1].split('.')[0]
    meta_df_file = meta_df_dir + file_name_without_path_or_ext + '.pkl'
    with open(meta_df_file, 'wb') as f:
        pickle.dump(meta_df, f)
    return meta_df_file


def process_pdf(meta_df_file, pdf_file,fields=None,ids=None, output_dir = './processed/'):
    
    # use meta data df to check ids
    with open(meta_df_file, 'rb') as f:
        meta_df = pickle.load(f)
    # get the pdfs 
    # lists to make pdf dataframe from 
    papers_ids_text = []; abstract = [] 
    body_text = []  # list of dicts (for each paper) with section, text, cite_spans
    whole_text = [] # list of strings (for each paper) of entire text in the body
    key_words = []  # key words mentioned with the abstract
    citations = [] 
    # if file is compressed
    if pdf_file[-3:] == '.gz':
        output_file = pdf_file[:-3]
        gz = gzip.open(pdf_file, 'rb')
        f = io.BufferedReader(gz)
        f_out = open(output_file,'wb')
    else:
        f = open(pdf_file)
        f_out = None
        
    for line in tqdm(f.readlines()):
        paper = json.loads(line)
        if paper['paper_id'] in meta_df['ids'].values: #ids defined in meta data df based on field
            if f_out: # if untaring
                f_out.write(line) # write that pdf (untarred pdf parse will only have selected papers)
            # get paper-id
            papers_ids_text.append(paper['paper_id'])
            abstract_text = ''
            terms = []
            # get abstract
            if paper['abstract']:
                abstract_text = paper['abstract'][0]['text']
                # get key_words in abstract
                if len(paper['abstract'])>1:   
                    if paper['abstract'][1]['text'][:11].lower() == 'index terms':
                        terms = paper['abstract'][1]['text'][12:].split(',') #remove "Index Terms-" or "INDEX TERMS " from string    
            abstract.append(abstract_text) 
            key_words.append(terms)
            # get sections and text from body
            text = []
            full_text = ''
            if paper['body_text']:
                for entry in paper['body_text']:
                    if entry['section'] and entry['text']:
                        section = {key: entry[key] for key in ['section', 'text', 'cite_spans']}
                        text.append(section)
                        if full_text: # why the if-else?
                            full_text = full_text + '\n' + entry['text']
                        else:
                            full_text = entry['text']
            body_text.append(text)
            whole_text.append(full_text)
            # get citations
            bib_entries = {}
            if paper['bib_entries']:
                bib_entries = paper['bib_entries']
            citations.append(bib_entries)
            
    f.close()
    if f_out:
        f_out.close()
    # create and save dataframe in output_dir/text_df
    text_df = pd.DataFrame({'paper_id': papers_ids_text, 'abstract': abstract,'key_words': key_words,'body_text': body_text, 'whole_text': whole_text,'citations':citations})
    text_df_dir = output_dir + 'text_df/'
    os.makedirs(text_df_dir, exist_ok=True)
    file_name_without_path_or_ext = pdf_file.split('/')[-1].split('.')[0]
    text_df_file = text_df_dir + file_name_without_path_or_ext + '.pkl'
    with open(text_df_file, 'wb') as f:
        pickle.dump(text_df, f)         
    return text_df_file       

In [43]:
def process_batch(metadata_file,pdf_file, fields=None, ids=None, output_dir = './processed/'):
    meta_df_file = process_metadata(metadata_file,fields,ids,output_dir)
    text_df_file = process_pdf(meta_df_file,pdf_file,fields,ids,output_dir)
    return meta_df_file,text_df_file


## Cleaning
Finer cleaning of data:
- remove stop words, high-frequency words, etc.

In [30]:
from nltk import pos_tag
from nltk.corpus import wordnet
def get_word_postag(word):
    #if pos_tag([word])[0][1].startswith('J'):
    #    return wordnet.ADJ
    #if pos_tag([word])[0][1].startswith('V'):
    #    return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        #return wordnet.NOUN
        return True
    else:
        return False
        #return wordnet.ADJ
        #return wordnet.NOUN

from nltk.tokenize import word_tokenize
# Preprocessing: tokenize words
def tokenize(text):
    return(word_tokenize(text))


def sent_to_words(sentences):
    for sentence in sentences:
        return(gensim.utils.simple_preprocess(str(sentence), min_len=3,deacc=True))  # deacc=True removes punctuations

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

# Preprocessing: remove stopwords
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords]) 
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Preprocessing: lemmatizing
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Preprocessing: remove short text
def find_longer_text(texts,k=200):
    return list(map(lambda x: len(x.split())>k,texts))
    
#     lengths = list(map(lambda x: len(x.split()), texts))
#     return [val >= k for val in lengths]
    #return [idx for idx, val in enumerate(lengths) if val >= k] 

# Preprocessing: alpha num
def keep_alphanum(words):
    #def isalphanum(word):
    #return word.isalnum()
    return filter(lambda word: word.isalnum(), words)
    #return [word for word in words if word.isalnum()]

# Preprocessing: keep nouns
def keep_nouns(words):
    return filter(get_word_postag, words)
    #return [word for word in words if get_word_postag(word) =='n']

# Preprocessing: keep words >= 3 in length
def keep_longer_words(words):
    return list(filter(lambda x: (len(x) >= 3), words))
    #return [word for word in words if len(word) >= 3]

# Preprocessing: lemmatize
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
def lemmatize(words):
    return list(map(lm.lemmatize, words)) 

# Preprocessing: stemming
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
def stemming(words):
    #return [ps.stem(word) for word in words]
    return map(ps.stem, words)

def remove_digits(words):
    return list(filter(lambda x: x.isalpha(), words))
    #return list(filter(lambda x: x.isalpha(), words))
#     return [word for word in words if word.isalpha()]

def merged(words):
    return ' '.join(word for word in words)        


In [31]:
def clean_sections(textdf, file_name):
    pass

In [32]:
def ids_to_keep(meta_df,fields):
    pass
def clean_pdf(text_df, file_name, output_dir):
    
    start = time.time()
    

    # if index is not paper_id
    if not text_df.index.name: 
        print('changing index to paper_id')
        text_df.set_index('paper_id')
        
    ids = text_df.index.values.astype(str)
    
        
    contents = text_df['whole_text'].values.tolist()
    abstracts = text_df['abstract'].values.tolist()
    
    # Add abstract to text
    contents = [i + j for i, j in zip(contents, abstracts)]
    
    t = time.time()
    print(t-start)
    
    # Remove new line characters
    contents = map(lambda x: re.sub('\s+', ' ', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: lower case text
    contents = map(lambda x: x.lower(),contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: keep alphanumeric
    contents = map(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stand along numbers
    contents = map(lambda x: re.sub(" \d+ ", " ", x), contents)

    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stop words
    contents = map(remove_stopwords, contents)
    
    t = time.time()
    print(t-start)
    
    contents = list(contents)

    # Preprocessing: remove short text
    inds = find_longer_text(contents)
    contents = itertools.compress(contents, inds)
    ids = list(itertools.compress(ids, inds))
    
    key_words = text_df.loc[ids]['key_words'].values
    print('Tokenizing')
    
    # Tokenize words + remove punctuation
    tokenized_contents = map(tokenize,contents) # documents in BOW format
#     word_list = [tokenize(article) for article in contents]
    t = time.time()
    print(t-start)
    
    # Remove numbers
    tokenized_contents = map(remove_digits, tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    # Keep longer words
#     word_list = [keep_longer_words(words) for words in  word_list]
    tokenized_contents = map(keep_longer_words,  tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    print('Lemmatizing')
    
    # Preprocessing: lemmatize
    tokenized_contents = map(lemmatize, tokenized_contents)
    
#     print(list(word_list)[0])
    
    t = time.time()
    print(t-start)
    
    print('Bag of Words Representation')
    tokenized_contents = list(tokenized_contents)

    dct = corpora.Dictionary(tokenized_contents) # make dct before corpus
#     doc2bow = partial(dct.doc2bow,allow_update=True)
    
    print('length of dct before filter_extreme: ', len(dct))
    dct.filter_extremes() # using default params # filter dct before creating corpus
    print('length of dct after filter_extreme: ', len(dct))
    

    # Make corpus after any changes to dct
    corpus = list(map(lambda x: dct.doc2bow(x,allow_update=True), tokenized_contents))    
#     corpus = [dct.doc2bow(doc, allow_update=True) for doc in word_list]

    t = time.time()
    print(t-start)


    
    word_list =  [item for sublist in tokenized_contents for item in sublist]
    counter=collections.Counter(word_list)
  
    os.makedirs(output_dir, exist_ok=True)
    output_file_name = output_dir + file_name + '_clean.pkl'

    
    with open(output_file_name, 'wb') as f:  # Python 3: open(..., 'wb') 
        d = {'dct': dct, 'corpus': corpus, 'docs': tokenized_contents, 
             'counter': counter, 'ids': ids, 'word_list':word_list,
             'key_words':key_words}
        pickle.dump(d, f)
    return d

In [17]:
user = "v"
if user == "v":
    sample_data_dir = "/Users/virenbajaj/Desktop/Columbia Fall 20/Graphical Models/project/20200705v1/sample/"
    full_data_dir = "/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/"
else:
    full_data_dir = '20200705v1/full/'
    
metadata_dir = full_data_dir + 'metadata/'
pdf_parses_dir = full_data_dir + 'pdf_parses/'

In [47]:
files = range(2,99)
metadata = [metadata_dir + f'metadata_{i}.jsonl.gz' for i in files]
pdfs = [pdf_parses_dir + f'pdf_parses_{i}.jsonl.gz' for i in files]
fields = []
get_ids = flat_links # from cell that gets links of all refs
print(metadata, pdfs)

['/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_2.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_3.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_4.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_5.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_6.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_7.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_8.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_9.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_10.jsonl.gz', '/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/metadata/metadata_11.jsonl.gz', '/Volumes/Extreme

In [13]:
# def add_citations(text_df_file, pdf_file):
#    'Add citation column to current DF by parsing pdf_parse'
#     with open(text_df_file, 'rb') as f:
#         text_df = pickle.load(f)
        
#     citations = []
#     with open(pdf_file, 'rb') as f:
#         for line in tqdm(f.readlines()):
#             paper = json.loads(line)
#             # get citations
#             bib_entries = {}
#             if paper['bib_entries']:
#                 bib_entries = paper['bib_entries']
#             citations.append(bib_entries)
#     text_df['citations'] = citations
    
#     with open(text_df_file, 'wb') as f:
#         pickle.dump(text_df, f)

In [46]:
# for batch in zip(metadata,pdfs):
#     process_batch(batch[0], batch[1], ids=get_ids)
for f in metadata:
    process_metadata(f,fields=None, ids=get_ids,output_dir = './processed/refs')
# process_pdf('processed/meta_df/metadata_0.pkl', pdfs[0],fields,output_dir='./processed/')
# add_citations(text_df_file='process_pdf(meta_df_file,pdf_file,fields,output_dir)',
#             pdf_file ='/Volumes/Extreme SSD/Library/SemanticScholar Data/20200705v1/full/pdf_parses/pdf_parses_0.jsonl')

100%|██████████| 1365929/1365929 [00:18<00:00, 73831.57it/s]


In [41]:
get_ids 77490118

['1000178',
 '10002174',
 '10002490',
 '10002794',
 '1000705',
 '10007445',
 '10010362',
 '10012695',
 '10013342',
 '10014509',
 '10014916',
 '10019125',
 '100192893',
 '1002231',
 '10024504',
 '10025155',
 '10028835',
 '10034146',
 '10034376',
 '10038278',
 '10044762',
 '1005033',
 '100505785',
 '10051618',
 '10054959',
 '100562543',
 '10056641',
 '10060377',
 '10060951',
 '10061678',
 '100697845',
 '10072679',
 '10072787',
 '10073547',
 '10077745',
 '10079396',
 '10081183',
 '1008597',
 '1008664',
 '10093314',
 '10094632',
 '10100203',
 '10101302',
 '1010254',
 '10108708',
 '10112350',
 '1011491',
 '10118629',
 '10120169',
 '1012360',
 '10124644',
 '10125953',
 '10126561',
 '10127893',
 '1012855',
 '10130384',
 '1013261',
 '10137687',
 '10138819',
 '10138819',
 '10142996',
 '10147449',
 '10147645',
 '10148298',
 '101494421',
 '10149856',
 '10152780',
 '10154359',
 '10156870',
 '10156891',
 '101582512',
 '10163399',
 '101635752',
 '1017340',
 '10174060',
 '10177970',
 '10180946',
 '10

In [5]:
with open('./processed/meta_df/metadata_0.pkl', 'rb') as f:
    meta_df = pickle.load(f)
with open('./processed/text_df/pdf_parses_0.pkl','rb') as f:
    text_df = pickle.load(f)

In [11]:
text_df = text_df.set_index('paper_id') # SET INDEX TO PAPER ID TO INDEX INTO DF
text_df.index.name 

None


'paper_id'

In [15]:
# filter DF based on fields
def field_in(df, selected):
    'Filter df based on fields in selected'
    def f(fields):
        return any([x in selected for x in fields])
    mask = list(map(f,df.field))
    return df[mask]
# med_meta_df = field_in(meta_df,['Medicine'])
# cs_meta_df = field_in(meta_df, ['Computer Science'])

In [6]:
# med_text_df = text_df.filter(items=med_meta_df.ids.values, axis=0)
# cs_text_df = text_df.filter(items=cs_meta_df.ids.values, axis=0)

In [19]:
key_words = text_df['key_words'][text_df.key_words.str.len() > 0 ] 
text_df_kw = text_df.loc[key_words.index] # papers with key words

In [24]:
def get_links(paper_refs):
    'Retreive the paper_ids (links) from refrences of the paper'
    refs, links = [], []
    for key,value in paper_refs.items():
        if value['link']:
            links.append(value['link'])
            refs.append(key)
    return links, refs

all_citations = (text_df_kw.citations.values)
all_links=[]
all_refs=[]
for citation in all_citations:
    links,refs = get_links(citation)
    all_links.append(links)
    all_refs.append(refs)

In [38]:
# text_df['refs'] = all_refs
# text_df['links'] = all_links

In [25]:
flat_links = [item for sublist in all_links for item in sublist]
links_found = set(flat_links).intersection(set(meta_df.ids)) # links in current df

In [26]:
len(links_found), len(flat_links)

(101, 13982)

In [15]:
file_name = 'pdf_parses_0_kw'
# for field in fields:
d = clean_pdf(text_df_kw,file_name,output_dir=f'./cleaned/cs-med/')

0.5264949798583984
0.5274271965026855
0.5274591445922852
0.5274801254272461
0.5274980068206787
0.5279140472412109
Tokenizing
4.992978096008301
4.992990016937256
4.993922233581543
Lemmatizing
4.994757890701294
Bag of Words Representation
length of dct before filter_extreme:  34400
length of dct after filter_extreme:  6920
16.1564302444458


In [16]:
cleaned_file= './cleaned/cs-med/pdf_parses_0_kw_clean.pkl'
with open(cleaned_file, 'rb') as f:
    cleaned_data = pickle.load(f)

In [17]:
dct = cleaned_data['dct']
corpus = cleaned_data['corpus']
counter = cleaned_data['counter']
ids = list(cleaned_data['ids'])
word_list = cleaned_data['word_list']
docs = cleaned_data['docs']

In [63]:
# documents = corpus.get_texts()
print(len(corpus))
print(len(list(ids)))
print(len(dct.dfs))
print(len(counter))
print(len(docs))
print(len(word_list))

493
493
34400
34400
493
1279903


## TODO


In [ ]:
# file_path = cwd + '\\Preprocessed\\'
# def process_pdf(file_name, batch_num, start_ind, end_ind, ids):
#     textdf = read_pdf(file_name, start_ind, end_ind, ids)
#     save_path = file_path + str(batchnum)
#     output = clean_pdf(textdf, save_path)
    

In [66]:
# def process_batch(batch_ind, batch_size=50000, field='Computer Science'):
#     file_name_meta = '20200705v1/full/metadata/metadata_' + str(batch_ind) + '.jsonl'
#     file_name_pdf = '20200705v1/full/pdf_parses/pdf_parses_' + str(batch_ind) + '.jsonl'
    
#     import os
#     cwd = os.getcwd()
#     file_path = cwd + '\\Preprocessed\\' 

#     start = time.time()
    
#     nlines = sum(1 for line in open(file_name_pdf))
#     batch_num = int(np.ceil(nlines / batch_size))
    
#     print('Processing metadata file', batch_ind)
#     selected_data = process_metadata(file_name_meta, field)
#     selected_ids = selected_data['ids'].values
    
#     with open(file_path+'metadata.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#         pickle.dump(selected_data, f)
    
#     t = time.time()
#     print(t-start)
    
#     for i in range(batch_num):
#         print('Processing pdfs batch Number: ', i)
#         line_nums = [batch_size*i, batch_size*(i+1)]
#         textdf = read_pdf(file_name_pdf,line_nums[0],line_nums[1], ids)  
        
#         t = time.time()
#         print(t-start)
        
#         print('Processing pdfs batch Number: ', i)

#         output = clean_pdf(textdf, file_path+str(i))
        
#         t = time.time()
#         print(t-start)

In [293]:
# process_batch(batch_ind=0, field='Computer Science')

In [132]:
# word_counts = sorted(dct.dfs.items(), key = lambda x: x[1], reverse=True)
# top_ids = [x[0] for x in word_counts[0:100]]
# top_words = [dct.id2token[x] for x in top_ids]

In [133]:
# dct.filter_tokens(bad_ids=top_ids)